In [1]:
import os
import vllm
import json
from easysteer.hidden_states import get_all_hidden_states
from vllm import LLM,SamplingParams

from easysteer.steer import extract_pca_control_vector,StatisticalControlVector


os.environ["CUDA_VISIBLE_DEVICES"] = "4"
os.environ["VLLM_USE_V1"] = "0"

file_path = "../../temp/test.jsonl" #MATH-500
# file_path = "../../temp/test.jsonl" #GSM8K

model_path = "/data/zju-46/shenyl/hf/model/deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B/"
model_path2 = "/data/zju-46/shenyl/hf/model/deepseek-ai/DeepSeek-R1-Distill-Qwen-7B/"


# vector_path = "vectors/thinking_switch_pca_MATH-500.gguf" #MATH-500
vector_path = "MATH500.gguf"

num_question = 50

/home/meixinyu/anaconda3/envs/easysteer/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO 07-15 15:18:07 [__init__.py:244] Automatically detected platform cuda.


In [2]:
problem_list = []

try:
    with open(file_path, 'r', encoding='utf-8') as f:
        line_count = 0
        for line in f:
            if line_count >= num_question:
                break
            stripped_line = line.strip()
            if not stripped_line:
                continue
                
            try:
                data = json.loads(stripped_line)
                if "problem" in data:
                    problem_list.append(data["problem"])
                    line_count += 1
                elif "question" in data:
                    problem_list.append(data["question"])
                    line_count += 1
            except json.JSONDecodeError:
                print(f"skip: {line[:50]}...")
                continue

except Exception as e:
    print(f"{str(e)}")
# problem_list = ["Find the roots of $(x - 3)^3 + (x -7)^3 = (2x - 10)^3.$","A regular hexagon can be divided into six equilateral triangles. If the perimeter of one of the triangles is 21 inches, what is the perimeter, in inches, of the regular hexagon?",""]

In [3]:
texts_fast = []
texts_slow = []

for prob in problem_list:
    texts_slow.append("<|User|>"+prob+"\n<|Assistant|><think>\n Let's think step by step.\n")

for prob in problem_list:
    texts_fast.append("<|System|>"
    +"You are an expert competition mathematician. "
    +"When you give a solution, express it **primarily in formal math notation"
    +"** with *minimal* surrounding English. "
    +"Return the final answer in a boxed format.\n"
    +"<|User|>"
    +"Solve the following problem step by step. "
    +"**Answer almost entirely in math notation**; keep English words to the bare minimum.\n\n"
    +"Problem:\n"+prob
    +"\n<|Assistant|><think>\n")

texts=texts_fast+texts_slow
llm = LLM(model=model_path2,task="generate",tensor_parallel_size=1,enforce_eager=True)

answers_fast = llm.generate(
    texts_fast,
    SamplingParams(
        temperature=0,
        max_tokens=4096,
        skip_special_tokens=False,
        repetition_penalty=1.1
    ),
)

answers_fast = [answer.outputs[0].text for answer in answers_fast]
qa_pairs_fast = [texts_fast[i] + answers_fast[i] for i in range(len(texts_fast))]




INFO 07-15 15:18:17 [config.py:1472] Using max model len 131072
WARNING 07-15 15:18:17 [arg_utils.py:1566] Chunked prefill is enabled by default for models with max_model_len > 32K. Chunked prefill might not work with some features or models. If you encounter any issues, please disable by launching with --enable-chunked-prefill=False.


2025-07-15 15:18:17,452	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


INFO 07-15 15:18:17 [config.py:2285] Chunked prefill is enabled with max_num_batched_tokens=2048.
WARNING 07-15 15:18:17 [cuda.py:102] To see benefits of async output processing, enable CUDA graph. Since, enforce-eager is enabled, async output processor cannot be used
INFO 07-15 15:18:17 [llm_engine.py:232] Initializing a V0 LLM engine (v0.1.dev7499+g2a4b294.d20250712) with config: model='/data/zju-46/shenyl/hf/model/deepseek-ai/DeepSeek-R1-Distill-Qwen-7B/', speculative_config=None, tokenizer='/data/zju-46/shenyl/hf/model/deepseek-ai/DeepSeek-R1-Distill-Qwen-7B/', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config={}, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=131072, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=True, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(backend='auto'

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  50% Completed | 1/2 [01:00<01:00, 60.20s/it]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [02:18<00:00, 70.72s/it]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [02:18<00:00, 69.14s/it]



INFO 07-15 15:20:38 [default_loader.py:272] Loading weights took 138.52 seconds
INFO 07-15 15:20:38 [model_runner.py:1255] Model loading took 14.2717 GiB and 138.692121 seconds
INFO 07-15 15:20:40 [worker.py:295] Memory profiling takes 1.04 seconds
INFO 07-15 15:20:40 [worker.py:295] the current vLLM instance can use total_gpu_memory (47.44GiB) x gpu_memory_utilization (0.90) = 42.69GiB
INFO 07-15 15:20:40 [worker.py:295] model weights take 14.27GiB; non_torch_memory takes 0.08GiB; PyTorch activation peak memory takes 1.43GiB; the rest of the memory reserved for KV Cache is 26.92GiB.
INFO 07-15 15:20:40 [executor_base.py:115] # cuda blocks: 31498, # CPU blocks: 4681
INFO 07-15 15:20:40 [executor_base.py:120] Maximum concurrency for 131072 tokens per request: 3.84x
INFO 07-15 15:20:44 [llm_engine.py:430] init engine (profile, create kv cache, warmup model) took 5.38 seconds


Processed prompts: 100%|██████████| 50/50 [01:56<00:00,  2.33s/it, est. speed input: 66.14 toks/s, output: 663.60 toks/s] 


In [6]:
del llm

llm = LLM(model=model_path,task="generate",tensor_parallel_size=1,enforce_eager=True)

answers_slow = llm.generate(
    texts_slow,
    SamplingParams(
        temperature=0,
        max_tokens=4096,
        skip_special_tokens=False,
        repetition_penalty=1.1
    ),
)
# texts=texts_fast+texts_slow


answers_slow = [answer.outputs[0].text for answer in answers_slow]
qa_pairs_slow = [texts_slow[i] + answers_slow[i] for i in range(len(texts_slow))]



INFO 07-15 15:25:05 [config.py:1472] Using max model len 131072
WARNING 07-15 15:25:05 [arg_utils.py:1566] Chunked prefill is enabled by default for models with max_model_len > 32K. Chunked prefill might not work with some features or models. If you encounter any issues, please disable by launching with --enable-chunked-prefill=False.
INFO 07-15 15:25:05 [config.py:2285] Chunked prefill is enabled with max_num_batched_tokens=2048.
WARNING 07-15 15:25:05 [cuda.py:102] To see benefits of async output processing, enable CUDA graph. Since, enforce-eager is enabled, async output processor cannot be used
INFO 07-15 15:25:05 [llm_engine.py:232] Initializing a V0 LLM engine (v0.1.dev7499+g2a4b294.d20250712) with config: model='/data/zju-46/shenyl/hf/model/deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B/', speculative_config=None, tokenizer='/data/zju-46/shenyl/hf/model/deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B/', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config={

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.39s/it]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.39s/it]


INFO 07-15 15:25:09 [default_loader.py:272] Loading weights took 1.53 seconds


INFO 07-15 15:25:09 [model_runner.py:1255] Model loading took 3.3148 GiB and 1.634453 seconds
INFO 07-15 15:25:10 [worker.py:295] Memory profiling takes 0.41 seconds
INFO 07-15 15:25:10 [worker.py:295] the current vLLM instance can use total_gpu_memory (47.44GiB) x gpu_memory_utilization (0.90) = 42.69GiB
INFO 07-15 15:25:10 [worker.py:295] model weights take 3.31GiB; non_torch_memory takes 0.00GiB; PyTorch activation peak memory takes 1.38GiB; the rest of the memory reserved for KV Cache is 38.00GiB.
INFO 07-15 15:25:10 [executor_base.py:115] # cuda blocks: 88932, # CPU blocks: 9362
INFO 07-15 15:25:10 [executor_base.py:120] Maximum concurrency for 131072 tokens per request: 10.86x
INFO 07-15 15:25:10 [llm_engine.py:430] init engine (profile, create kv cache, warmup model) took 1.12 seconds


Processed prompts: 100%|██████████| 50/50 [01:22<00:00,  1.65s/it, est. speed input: 53.51 toks/s, output: 539.60 toks/s] 


In [7]:
del llm
llm = LLM(model=model_path,task="reward",tensor_parallel_size=1)

hidden_states_fast, _= get_all_hidden_states(llm, qa_pairs_fast)
hidden_states_slow, _= get_all_hidden_states(llm, qa_pairs_slow)
all_hidden_states=hidden_states_fast+hidden_states_slow

INFO 07-15 15:26:49 [config.py:1472] Using max model len 16384
INFO 07-15 15:26:49 [config.py:4615] Only "last" pooling supports chunked prefill and prefix caching; disabling both.
INFO 07-15 15:26:49 [llm_engine.py:232] Initializing a V0 LLM engine (v0.1.dev7499+g2a4b294.d20250712) with config: model='/data/zju-46/shenyl/hf/model/deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B/', speculative_config=None, tokenizer='/data/zju-46/shenyl/hf/model/deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B/', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config={}, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=16384, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(backend='auto', disable_fallback=False, disable_any_whitespace=False, disable_additional_propertie

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  1.01it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  1.01it/s]


INFO 07-15 15:26:52 [default_loader.py:272] Loading weights took 1.09 seconds


INFO 07-15 15:26:53 [model_runner.py:1255] Model loading took 2.8793 GiB and 1.197712 seconds


Processed prompts: 100%|██████████| 50/50 [00:05<00:00,  8.99it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]


In [8]:
from easysteer.steer import extract_diffmean_control_vector
control_vector = extract_diffmean_control_vector(
    all_hidden_states=all_hidden_states,
    positive_indices=list(range(num_question)), 
    negative_indices=list(range(num_question,2*num_question)),
    model_type="qwen2.5",
    token_pos=-1,
    normalize=False
)

Computing DiffMean directions: 100%|██████████| 28/28 [00:00<00:00, 8588.60it/s]


In [9]:
control_vector.export_gguf(vector_path)
control_vector = StatisticalControlVector.import_gguf(vector_path)
print(control_vector)

StatisticalControlVector(model_type='qwen2.5', method='diffmean', directions={0: memmap([-0.37762213,  0.3621289 ,  0.07671386, ...,  0.09187746,
         0.01702148, -0.02408692], shape=(1536,), dtype=float32), 1: memmap([-0.37708008,  0.3185938 ,  0.00083008, ...,  0.10798828,
        -0.12316895,  0.07854492], shape=(1536,), dtype=float32), 2: memmap([-0.50412107,  0.2696436 , -0.17655273, ...,  0.13890624,
        -0.12346679,  0.2603125 ], shape=(1536,), dtype=float32), 3: memmap([-0.6581836 ,  0.06841797, -0.3773291 , ...,  0.09156249,
        -0.11788085,  0.07102537], shape=(1536,), dtype=float32), 4: memmap([-0.67054677, -0.11105469, -0.1010547 , ..., -0.02114257,
        -0.01477537, -0.14418948], shape=(1536,), dtype=float32), 5: memmap([-0.7164061 , -0.07879883, -0.20362793, ...,  0.02737793,
        -0.156875  ,  0.0581836 ], shape=(1536,), dtype=float32), 6: memmap([-0.2943945 , -0.01407228, -0.28964844, ...,  0.01804686,
         0.31013674, -0.04800782], shape=(1536,), 